In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class BasicConv2d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(
            in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_planes, eps=0.001)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

## Inception

![](assets/inception.png)

#### examples

In [5]:
class InceptionBlock(nn.Module):
    def __init__(self, in_planes: int = 192):
        super().__init__()
        self.branch1 = BasicConv2d(in_planes, 64, (1, 1), 1, 0)
        self.branch2 = nn.Sequential(BasicConv2d(in_planes, 96, (1, 1), 1), BasicConv2d(96, 128, (3, 3), 1, 1))
        self.branch3 = nn.Sequential(BasicConv2d(in_planes, 16, (1, 1), 1), BasicConv2d(16, 32, (5, 5), 1, 2))
        self.branch4 = nn.Sequential(nn.MaxPool2d((3, 3), 1, 1), BasicConv2d(in_planes, 32, (1, 1), 1, 0))
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        output = torch.cat((x1, x2, x3, x4), dim=1)
        return output 
    

In [6]:
model = InceptionBlock(192)

In [7]:
batch_size = 4
features = torch.randn(batch_size, 192, 28, 28)
output = model(features)
print(f"Input shape : {features.shape}\nOutput shape : {output.shape}")

Input shape : torch.Size([4, 192, 28, 28])
Output shape : torch.Size([4, 256, 28, 28])


#### practice

#### practice 

1. V1
    * input_shape = (batch_size, 64, 28, 28)
    * output_shape = (batch_size, 128, 28, 28)
    * 4 branches, must have kernel (1, 1), (3, 3), (5, 5), (7, 7)


2. V3
    * input_shape = (batch_size, 64, 28, 28)
    * output_shape = (batch_size, 128, 28, 28)
    * 4 branches, must have kernel (1, 1), (3, 1), (1, 3), (5, 1), (1, 5)

In [8]:
features = torch.randn(batch_size, 64, 28, 28)

---

## ResNet - Residual Block

![](assets/residual.png)

#### examples

In [9]:
class ResidualBlock(nn.Module):
    def __init__(self, in_planes: int, out_planes: int, stride: int = 1):
        super().__init__()
        self.conv1 = BasicConv2d(in_planes, out_planes, (3, 3), stride, 1)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=(3, 3), stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)
        if (in_planes != out_planes) or (stride != 1):
            self.map = nn.Conv2d(in_planes, out_planes, kernel_size=(1, 1), stride=stride, padding=0, bias=False)
        else:
            self.map = nn.Identity()
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = self.map(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.bn(x)
        x += identity
        x = self.relu(x)
        return x 

In [10]:
model = ResidualBlock(64, 64, 1)

In [11]:
batch_size = 4
features = torch.randn(batch_size, 64, 32, 32)
output = model(features)
print(f"Input shape : {features.shape}\nOutput shape : {output.shape}")

Input shape : torch.Size([4, 64, 32, 32])
Output shape : torch.Size([4, 64, 32, 32])


#### v2

![](assets/r1_r2.png)

In [12]:
class ResidualBlockV2(nn.Module):
    def __init__(self, in_planes: int, out_planes: int, stride: int = 1):
        super().__init__()
        squeeze = out_planes // 4
        self.conv1 = BasicConv2d(in_planes, squeeze, (1, 1), 1, 0) ## squeeze
        self.conv2 = BasicConv2d(squeeze, squeeze, (3, 3), stride=stride, padding=1) ## squeeze
        self.conv3 = nn.Conv2d(squeeze, out_planes, (1, 1), 1, 0) ## expand
        if (in_planes != out_planes) or (stride != 1):
            self.map = nn.Conv2d(in_planes, out_planes, kernel_size=(1, 1), stride=stride, padding=0, bias=False)
        else:
            self.map = nn.Identity()
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = self.map(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x += identity
        return x 

In [13]:
model = ResidualBlockV2(64, 64, 1)

In [14]:
batch_size = 4
features = torch.randn(batch_size, 64, 32, 32)
output = model(features)
print(f"Input shape : {features.shape}\nOutput shape : {output.shape}")

Input shape : torch.Size([4, 64, 32, 32])
Output shape : torch.Size([4, 64, 32, 32])


#### practice
* combine Inception with Residual

![](assets/inceptionv4.png)

In [15]:
batch_size = 4
features = torch.randn(batch_size, 32, 32, 32)